In [1]:
import os 
import openai 
from dotenv import load_dotenv 

load_dotenv() 

openai.api_key = os.getenv("OPENAI_API_KEY")  
model_name = "gpt-3.5-turbo"

In [37]:
import json 

def get_current_weather(location, unit='farenheit'): 
    weather_info = {
        "location": location, 
        "temperature": "72", 
        "unit": unit, 
        "forecast": ["sunny", "windy"]
    } 
    return weather_info


In [5]:
# Define a function 
functions = [
    {
        "name": "get_current_weather", 
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string", 
                    "description": "The city and state", 
                },
                "unit": {"type": "string", "enum": ["celsius", "farenheit"]},
            },
            "required": ["location"]
        }
    }
]

In [6]:
# Message
messages = [
    {
        "role": "user",
        "content": "What's the weather in Boston?"
    }
]

In [8]:
response = openai.chat.completions.create(
    model=model_name, 
    messages=messages,
    functions=functions)
response

ChatCompletion(id='chatcmpl-9s1Cv4Ynb2sXRuCkY4eYznwjrRLBO', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Boston"}', name='get_current_weather'), tool_calls=None))], created=1722661381, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=15, prompt_tokens=74, total_tokens=89))

In [15]:
response.choices[0].message.content

In [23]:
function_call = response.choices[0].message.function_call 
arguments = function_call.arguments

In [24]:
# Unrelated 
messages = [
    {
        "role": "user", 
        "content": "Hi"
    }
]

In [29]:
# Function Call: 
# Auto - llm chooses what to do 
# None - llm is forced not to use functions 
# {"name": function_name} forces to call the specific function 
response = openai.chat.completions.create(
    model=model_name, 
    messages=messages,
    functions=functions,
    function_call='auto' 
)

In [26]:
response

ChatCompletion(id='chatcmpl-9s1GkvKhONOuiSuYVWkqb18SpCyTP', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1722661618, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=10, prompt_tokens=68, total_tokens=78))

In [62]:
# Passing back the arguments 
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
] 

response = openai.chat.completions.create(
    model=model_name, 
    messages=messages,
    functions=functions, 
    function_call={"name": "get_current_weather"}
)
response

ChatCompletion(id='chatcmpl-9s1lydTLniZJxovITQZZO5Kh6Sqkg', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Boston"}', name='get_current_weather'), tool_calls=None))], created=1722663554, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=5, prompt_tokens=85, total_tokens=90))

In [63]:
response_message = response.choices[0].message 
messages.append(response_message)

In [53]:
messages

[{'role': 'user', 'content': "What's the weather like in Boston?"},
 {'content': None,
  'role': 'assistant',
  'function_call': {'arguments': '{"location":"Boston"}',
   'name': 'get_current_weather'}}]

In [64]:
response_message

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Boston"}', name='get_current_weather'), tool_calls=None)

In [54]:
arguments = json.loads(response_message.function_call.arguments) 
observation = get_current_weather(arguments)  

In [65]:
# Add function call to messages (so llm knows history) 
messages.append({
    "role": "function",
    "name": "get_current_weather", 
    "content": observation
}) 

In [66]:
messages

[{'role': 'user', 'content': "What's the weather like in Boston?"},
 ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Boston"}', name='get_current_weather'), tool_calls=None),
 {'role': 'function',
  'name': 'get_current_weather',
  'content': {'location': {'location': 'Boston'},
   'temperature': '72',
   'unit': 'farenheit',
   'forecast': ['sunny', 'windy']}}]